In [20]:
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [21]:
# conn = MongoClient(host='127.0.0.1', port=27017)  # 实例化MongoClient
# db = conn.get_database('The_Wandering_Earth')  # 连接到TWH1数据库

# 猫眼数据以及保存为data.csv文件，无需从数据库中导出

# douban = db.get_collection('douban') # 连接到集合comment
# maoyan = db.get_collection("maoyan")
# douban_data = douban.find()
# maoyan_data = maoyan.find()

In [22]:
# data1 = pd.json_normalize([comment for comment in douban_data])
# data2 = pd.json_normalize([comment for comment in maoyan_data])       #以数据库打开


data1 = pd.read_csv("douban.csv")
data2 = pd.read_csv("data.csv")

In [23]:
data1.info()
print("\n")
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   _id        500 non-null    object
 1   comment    500 non-null    object
 2   id         500 non-null    object
 3   isWatched  500 non-null    object
 4   time       500 non-null    object
 5   rating     500 non-null    int64 
 6   likes      500 non-null    int64 
 7   city       452 non-null    object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102584 entries, 0 to 102583
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   content     102583 non-null  object 
 1   gender      102580 non-null  float64
 2   id          102580 non-null  float64
 3   nick        102579 non-null  object 
 4   replyCount  102580 non-null  float64
 5   score       102580 non-null  float64
 6   t

In [24]:
# 数据清洗
data1 = data1.drop(columns= "_id")
data1 = data1.drop(columns= "isWatched")
data1['time'] = pd.to_datetime(data1['time'], format='%Y-%m-%d')
data1 = data1[data1['time']>=pd.to_datetime('2019-02-01 00:00:00')]
data1.set_index(data1["time"], inplace=True)

print("\n")

data2 = data2.drop(columns='id')
data2 = data2.drop_duplicates(subset='userId')
data2['time'] = pd.to_datetime(data2['time'])

# 若使用数据库打开数据，数据时间字段的清洗用下面语句，将上一条语句注释
# data2['time'] = pd.to_datetime(data2['time']/1000, unit='s')

data2 = data2[data2['time']>=pd.to_datetime('2019-02-01 00:00:00')]
data2.set_index(data2["time"], inplace=True)
data1.info()
data2.info()



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 447 entries, 2019-02-05 to 2019-02-09
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   comment  447 non-null    object        
 1   id       447 non-null    object        
 2   time     447 non-null    datetime64[ns]
 3   rating   447 non-null    int64         
 4   likes    447 non-null    int64         
 5   city     403 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 24.4+ KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 102580 entries, 2019-02-11 10:10:00 to 2019-02-11 01:40:00
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   content     102579 non-null  object        
 1   gender      102580 non-null  float64       
 2   nick        102579 non-null  object        
 3   replyCount  102580 non-null  float64       
 4   sc

#### 一、数据详情
- 共有500条数据；
- 包含字段：
1.豆瓣：ID、评论内容、评论者昵称、是否观看过、评论时间、评分、评论点赞数、评论者所在城市
2.猫眼：评论内容、性别、评论ID、评论者昵称、回复数量、评分、时间、点赞数量、评论者ID、评论者等级
二、问题
1. 总体评价如何？
2. 总体评价的时间走向如何？
3. 高分的评价理由是什么？
4. 低分的评价理由是什么？
5. 低分的人群有哪些特征？（性别、等级）
6. 低分跟哪位演员有关？

 1. 总体评价如何？


In [25]:
print(data1['rating'].mean())   # 3.382
print(data2["score"].mean())


score_total1 = data1['rating'].value_counts().sort_index()
score_total2 = data2['score'].value_counts().sort_index()
print(score_total1)
print(score_total2)


3.302013422818792
9.241450575160851
1     43
2     51
3    164
4    106
5     83
Name: rating, dtype: int64
0.0       130
1.0      2329
2.0       429
3.0       481
4.0       359
5.0      1549
6.0      1343
7.0      2478
8.0      5810
9.0     14427
10.0    73245
Name: score, dtype: int64


In [26]:
from pyecharts.charts import Pie
from pyecharts.charts import Bar, Grid, Line
from pyecharts import options as opts
from pyecharts.components import Table


# 豆瓣：力荐=5，推荐=4，还行=3，较差=2，很差=1
# 猫眼：力荐=9-10，推荐=7-8，还行=5-6，较差=2-4，很差=1-0
rating_str = ["很差", "较差", "还行", "推荐", "力荐"]
rating_score1 = []
rating_score2 = [0,0,0,0,0]

# print(pd.value_counts(data1["rating"].values))

for i in range(1,6):
    rating_score1.append(pd.value_counts(data1["rating"].values)[i])

# print(pd.value_counts(data2["score"]))
for key,value in pd.value_counts(data2["score"]).items():
    # print(key,value)
    if key == 10 or key ==9:
        rating_score2[4] += value
    if key == 8 or key ==7:
        rating_score2[3] += value
    if key == 6 or key ==5:
        rating_score2[2] += value
    if key == 4 or key ==3:
        rating_score2[1] += value
    if key == 2 or key ==1:
        rating_score2[0] += value


#[['很差', 43], ['较差', 55], ['还行', 172], ['推荐', 128], ['力荐', 102]]
list_1 = [list(z) for z in zip(rating_str, rating_score1)]
list_2 = [list(z) for z in zip(rating_str, rating_score2)]
# print(list_2)
# print(list_1)

In [27]:
# 展示豆瓣中评分饼图
c1 = (
    Pie()
    .add(
        "",
        [['很差', 43], ['较差', 55], ['还行', 172], ['推荐', 128], ['力荐', 102]],
        center=["30%", "45%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}  {per|{d}%}  ",
            background_color="#eee",
            border_color="#aaa",
            border_width=1,
            border_radius=4,
            rich={
                "a": {"color": "#999", "lineHeight": 22, "align": "center"},
                "abg": {
                    "backgroundColor": "#e3e3e3",
                    "width": "100%",
                    "align": "right",
                    "height": 22,
                    "borderRadius": [4, 4, 0, 0],
                },
                "hr": {
                    "borderColor": "#aaa",
                    "width": "100%",
                    "borderWidth": 0.5,
                    "height": 0,
                },
                "b": {"fontSize": 16, "lineHeight": 33},
                "per": {
                    "color": "#eee",
                    "backgroundColor": "#334455",
                    "padding": [2, 4],
                    "borderRadius": 2,
                },
            },
        ),
    )
    .add(
        "",
        list_2,
        center=["65%", "45%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}  {per|{d}%}  ",
            background_color="#eee",
            border_color="#aaa",
            border_width=1,
            border_radius=4,
            rich={
                "a": {"color": "#999", "lineHeight": 22, "align": "center"},
                "abg": {
                    "backgroundColor": "#e3e3e3",
                    "width": "100%",
                    "align": "right",
                    "height": 22,
                    "borderRadius": [4, 4, 0, 0],
                },
                "hr": {
                    "borderColor": "#aaa",
                    "width": "100%",
                    "borderWidth": 0.5,
                    "height": 0,
                },
                "b": {"fontSize": 16, "lineHeight": 33},
                "per": {
                    "color": "#eee",
                    "backgroundColor": "#334455",
                    "padding": [2, 4],
                    "borderRadius": 2,
                },
            },
        ),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Rating_Pie"),
        legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="2%"),
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    .render("./Charts_html/Rating_Pie.html")
)
print(1)

1


In [28]:
list_ = [43, 55, 172, 128, 102]
# 展示豆瓣中评分折线图
bar = (
    Bar()
    .add_xaxis(rating_str)
    .add_yaxis("豆瓣", list_)
    .add_yaxis("猫眼", rating_score2)
    .set_global_opts(title_opts=opts.TitleOpts(title="评分分布柱状图"))
)
line = (
    Line()
    .add_xaxis(rating_str)
    .add_yaxis("豆瓣", list_)
    .add_yaxis("猫眼", rating_score2)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="评分分布折线图", pos_top="48%"),
        legend_opts=opts.LegendOpts(pos_top="48%"),
    )
)

grid = (
    Grid()
    .add(bar, grid_opts=opts.GridOpts(pos_bottom="60%"))
    .add(line, grid_opts=opts.GridOpts(pos_top="60%"))
    .render("./Charts_html/Rating_line_bar.html")
)

print(1)

1


##### 2. 总体评价的时间走向如何？

In [29]:

score_by_time1 = data1['rating'].resample('D').mean()
score_by_time2 = data2['score'].resample('D').mean()



douban_time = []
douban_score = []
for time,score in score_by_time1.items():
    douban_time.append(time)
    douban_score.append(score)


maoyan_time = []
maoyan_score = []
for time,score in score_by_time2.items():
    maoyan_time.append(time)
    maoyan_score.append(score)

# print(score_by_time1)
# print(score_by_time2)


c1 = (
    Bar()
    .add_xaxis(maoyan_time)
    .add_yaxis("猫眼评论", maoyan_score)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="猫眼评分随时间变化图"),
        datazoom_opts=[opts.DataZoomOpts()],
    )
    .render("./Charts_html/Score_by_time_maoyan.html")
)

c2 = (
    Bar()
    .add_xaxis(douban_time)
    .add_yaxis("豆瓣评论", douban_score)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="豆瓣评分随时间变化图"),
        datazoom_opts=[opts.DataZoomOpts()],
    )
    .render("./Charts_html/Score_by_time_douban.html")
)

print(1)




1


In [30]:
from pyecharts import options as opts
from pyecharts.charts import Map
from pyecharts.faker import Faker
import re

pattern = re.compile('[湖南|湖北|广东|广西|河南|河北|山东|山西|江苏|浙江|江西|黑龙江|'
                     '新疆|云南|贵州|福建|吉林|安徽四川|西藏|宁夏|辽宁|青海|'
                     '甘肃|陕西|内蒙古||台湾|北京|上海|天津]+')


info = data1['city'].value_counts()
province = info.keys()
amount = info.values

result = []

provinces = ['湖南', '湖北', '广东', '广西', '河南', '河北', '山东', '山西', '江苏', '浙江', '江西', '黑龙江', '新疆', '云南', '贵州', '福建', '吉林', '安徽', '四川', '西藏', '宁夏', '辽宁', '青海', '甘肃', '陕西', '内蒙古', '台湾', '北京', '上海', '天津']
temp_amount = []
temp_province = []


for i in range(len(province)):
    if pattern.findall(province[i]) != []:
        if pattern.findall(province[i])[0][:2] in temp_province:
            index = temp_province.index(pattern.findall(province[i])[0][:2])
            temp_amount[index] += int(amount[i])
        else:
            temp_province.append(pattern.findall(province[i])[0][:2])
            temp_amount.append(int(amount[i]))
        # result.append(temp)

result = [list(z) for z in zip(temp_province, temp_amount)]


c = (
    Map()
    .add("评论人数", result, "china")
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(max_=150, is_piecewise=True),
        title_opts=opts.TitleOpts(title="评论人数分布图"),
    )
    .render("./Charts_html/Map_amount.html")
)


<ipython-input-30-56fd84a7cc52>:6: FutureWarning: Possible set union at position 79
  pattern = re.compile('[湖南|湖北|广东|广西|河南|河北|山东|山西|江苏|浙江|江西|黑龙江|'


In [31]:
# 绘图需要导入的包

from pyecharts.charts import Scatter
from pyecharts.charts import Grid, Liquid, Page, Tab
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ThemeType

In [32]:
rating = data1['rating'].values
likes = data1['likes'].values
likes_new = [0, 0, 0, 0, 0]
sum = 0


# 按评分统计点赞人数
for i in range(len(rating)):
    if int(rating[i]) == 1:
        likes_new[0] += int(likes[i])
    elif int(rating[i]) == 2:
        likes_new[1] += int(likes[i])
    elif int(rating[i]) == 3:
        likes_new[2] += int(likes[i])
    elif int(rating[i]) == 4:
        likes_new[3] += int(likes[i])
    else:
        likes_new[4] += int(likes[i])

for j in range(len(likes_new)):
    sum += likes_new[j]

favorate_rate = (likes_new[3] + likes_new[4]) / sum
poor_rate = (likes_new[0] + likes_new[1]) / sum
# print(type(favorate_rate), type(poor_rate))

# print(likes_new)
def scatter_rating_likes() -> Scatter:
    c = (
        Scatter()
        .add_xaxis(range(1,6))
        .add_yaxis("点赞数", likes_new)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="评分-点赞图"),
            visualmap_opts=opts.VisualMapOpts(type_="size", max_=100000, min_=20000),
        )
    )
    return c

def grid_liquid_favorate_rate() -> Grid:
    l1 = (
        Liquid().
        add(
        "好评率",
        [favorate_rate],
        center=["60%", "50%"],
        label_opts=opts.LabelOpts(
            font_size=50,
            formatter=JsCode(
                """function (param) {
                        return (Math.floor(param.value * 10000) / 100) + '%';
                    }"""
             ),
            position="inside",
            ),
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="好评率和差评率"))
    )

    l2 = (
        Liquid().
        add(
        "差评率",
        [poor_rate],
        center=["25%", "50%"],
        label_opts=opts.LabelOpts(
            font_size=50,
            formatter=JsCode(
                """function (param) {
                        return (Math.floor(param.value * 10000) / 100) + '%';
                    }"""
             ),
            position="inside",
            ),
        )
    )

    grid = Grid().add(l1, grid_opts=opts.GridOpts()).add(l2, grid_opts=opts.GridOpts())
    return grid

# grid.render("./charts_html/scatter_rating_likes.html")
def page_layout():
    page = Page()
    page.add(
        scatter_rating_likes(),
        grid_liquid_favorate_rate()
    )
    page.render("./Charts_html/Rating_likes.html")

if __name__ == "__main__":
    page_layout()

### 文本情感分析：
##### 3. 高分的评价理由是什么？

In [33]:
import jieba
from collections import Counter
from pyecharts.charts import WordCloud
from pyecharts.globals import SymbolType

jieba.add_word('屈楚萧')
jieba.add_word('刘启')
jieba.add_word('吴京')
jieba.add_word('刘培强')
jieba.add_word('李光洁')
jieba.add_word('王磊')
jieba.add_word('吴孟达')
jieba.add_word('达叔')
jieba.add_word('韩子昂')
jieba.add_word('赵今麦')
jieba.add_word('韩朵朵')

swords = [x.strip() for x in open ('stopwords.txt', encoding="utf-8")]

print(swords)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lLate\AppData\Local\Temp\jieba.cache
Loading model cost 0.814 seconds.
Prefix dict has been built successfully.


['则', '抑或', '次第', '反而', '即令', '好', '怎么样', '｝', '［①ａ］', '＃', '此后', '呢', '接着', '呆呆地', 'ｂ］', '嘎', '）、', '假如', '当下', '乎', '［②Ｂ］', '个', '那边', 'ｎｇ昉', '不起', '乃', '自后', '其后', '老是', '其他', '充其极', '要不', '［①ｉ］', '至', '不免', '孰知', '来不及', '到了儿', '非但', '管', '起先', '既', '非独', '如若', '奋勇', '还要', '哎', '因了', '[', '何处', '自', '哟', '较比', '传闻', '于是乎', '对于', '反过来', '别管', '-', '［①①］', '宁', '［⑤ｆ］', '替', '＞λ', '接下来', '莫非', '各', '非常', '而论', '起初', '矣乎', '以便', '②ｃ', '请勿', '｝＞', '大体上', '将要', '关于', '比起', '起', '赶早不赶晚', '⑩', '.一', '断然', '何乐而不为', '有', '分别', '受到', '咋', '∈［', '不得不', '［②Ｇ］', '......', '瑟瑟', '及至', '大家', '莫如', '只要', '具体说来', '［④ｃ］', '较为', '———', '不限', '.日', '多少', '毕竟', '所幸', '比', '不论', '只当', '）', '是以', '吓', '［］', '每当', '。', '决不', '何以', '更为', '顷刻', '如常', '自家', '近几年来', '⑨', '长话短说', '咱', '［③ｄ］', '绝顶', '千万千万', '并排', '到头来', '’', '一切', '极度', '［⑤ａ］', '仍然', '全年', '如何', '纵令', '－', '～＋', '从古到今', '嘛', '大抵', '难得', '但', '咱们', '［②⑤］', '当然', '除此而外', '她是', '假使', '［④ｄ］', '这次', '综上所述', '难说', '［③ｇ］', ';', '从头', '方才', '理应', '曾', '接

In [34]:
import re
pattern = re.compile('[^\d]+')

def plot_word_cloud_maoyan(data, swords, path):
    # print(data['content'])
    # 2019-02-11 10:10:00
    text = ''
    # print(type(data['content'].values[99]), data['content'].values[299])
    # 将所有评论连接起来

    for temp_i in data['content']:
        text += str(temp_i)

    words = list(jieba.cut(text))
    ex_sw_words = []
    for word in words:
        if len(word)>1 and (word not in swords):
            ex_sw_words.append(word)
    c = Counter()
    c = Counter(ex_sw_words)
    wc_data = pd.DataFrame({'word':list(c.keys()), 'counts':list(c.values())}).sort_values(by='counts', ascending=False).head(100)
    # print(wc_data)

    c = (
    WordCloud()
    .add("", wc_data.values, word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title=path))
    .render("./Charts_html/Maoyan_" + path + ".html")
    )


def plot_word_cloud_douban(data, swords, path):
    text = ''.join(data['comment'])
    words = list(jieba.cut(text))
    ex_sw_words = []
    for word in words:
        if len(word)>1 and (word not in swords):
            ex_sw_words.append(word)
    c = Counter()
    c = Counter(ex_sw_words)
    wc_data = pd.DataFrame({'word':list(c.keys()), 'counts':list(c.values())}).sort_values(by='counts', ascending=False).head(100)
    print(wc_data)

    c = (
    WordCloud()
    .add("", wc_data.values, word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title=path))
    .render("./Charts_html/Douban_" + path + ".html")
    )

In [35]:
# 绘制豆瓣的评价的词云
# 高分
plot_word_cloud_douban(data=data1[data1['rating']>=4], swords=swords, path="Worldcloud_favorate")
# 低分
plot_word_cloud_douban(data=data1[data1['rating']<=2], swords=swords, path="Worldcloud_poor")

     word  counts
32     中国     123
16     电影      93
40     科幻      91
25     地球      78
26     流浪      45
..    ...     ...
662    解决       8
319    未来       8
383  瑕不掩瑜       8
138    漫游       8
270    缺点       8

[100 rows x 2 columns]
     word  counts
151    电影      49
4      地球      35
41     中国      33
78     科幻      28
144    特效      28
..    ...     ...
306    电梯       4
188  莫名其妙       4
998    太多       4
350    大片       4
281    处理       4

[100 rows x 2 columns]


In [36]:
# 豆瓣高分的部分评论
data1[data1['rating']>3]

,comment,id,time,rating,likes,city
time,,,,,,
2019-02-04,\n“北京道路安全委提醒你：道路千万条，安全第一条，行车不规范，亲人两行泪” 这句广播语真是...,影志,2019-02-04,4,71948,北京
2019-02-02,\n中国导演能拍出这样的硬科幻，看到这样的完成度，想想就激动，可以让人原谅一切不完美！说这部...,亵渎电影,2019-02-02,4,9120,北京
2019-02-06,\n8.8，热评第一条我就呵呵了，中国科幻电影为什么不值得我们打高分去鼓励，非要放在国际水平...,舒服Kimodi,2019-02-06,5,9824,辽宁大连
2019-02-05,\n观影过程中的情绪很复杂，一方面感叹特效的用心和部分设定的精妙，另一方面又很讨厌导演极尽谄...,二十二岛主,2019-02-05,4,3460,北京
2019-02-05,\n春节期间最大的一场烟花，是看你爸为你和地球点燃的宇宙爆炸，太朋克了。\n,姨妈的鸭,2019-02-05,5,6827,上海
...,...,...,...,...,...,...
2019-02-07,\n中国科幻电影有希望了，老泪纵横啊！就是演的太像“演的”了！两个小孩的戏，还有姥爷的配音有...,莫亚,2019-02-07,5,33,北京
2019-02-05,\n看完以后，中国科幻电影元年不元年，中国科幻有没有希望，这种谁还在乎。仅说电影，已经完全值...,代号273,2019-02-05,5,22,北京
2019-02-05,\n改编赞，特效赞，情感冲击到最后也非常赞。小说的主人公没见过星星，现实中的观众没见过中国硬...,我爱罗宾,2019-02-05,4,35,北京


In [37]:
# 豆瓣低分的部分评论
data1[data1['rating']<3]

,comment,id,time,rating,likes,city
time,,,,,,
2019-02-05,\n求求编剧，人类都快失去地球了，生存才是最重要的。请别再bb这点家庭琐事和煽情烂梗。白瞎了...,说给自己听,2019-02-05,1,14487,上海
2019-02-05,\n台词做作剧情薄弱没一个演员在线 无时无刻的尬笑走心台词还以为编剧是激素泛滥的中年少女 感...,gus,2019-02-05,2,14653,广东广州
2019-02-05,\n一分。画面分。\n在一个烧石头都能做热核聚变的高科技时代，会搞不清楚发动机的工作条件导致...,淺潛,2019-02-05,1,6474,江苏南京
2019-02-05,\n这个设定本身带来的那种悲壮甚至一丝浪漫感全被各种毫无节制的煽情和空到不行的台词给毁了 ★...,qw0aszx,2019-02-05,2,7022,浙江杭州
2019-02-05,\n新时代特色社会主义硬核科幻(科幻的核心在于探索发问，而不是去解释回答，这样的作品属于玄幻)\n,BeAst,2019-02-05,2,6054,陕西西安
...,...,...,...,...,...,...
2019-02-12,\n特效很棒，美而绝望。但作为电影，故事情节单薄，没有冲突点，所以整个片子一点都不吸引人，就...,鱼二度,2019-02-12,2,17,广东广州
2019-02-13,\n刻意煽情让人感觉十分难受且尴尬，另有多处硬伤经不起推敲。一定要说是中国科幻电影的崛起，个...,大河马,2019-02-13,2,17,NaN
2019-02-08,\n全片就很尬。剧情发展和突然的煽情，整个尬到最后；人物设定也很奇怪，全片找不到任何一个爆满...,王强尼乐百事,2019-02-08,2,39,浙江杭州


In [38]:
# 豆瓣高分评论
# 数量过多，需要解除注释即可查看
# for i in data1[data1['rating']>3]['comment']:
#     print(i+'\n')

In [39]:
# 豆瓣低分评论
# for i in data1[data1['rating']<3]['comment']:
#     print(i+'\n')

In [40]:
# 绘制猫眼的评价的词云
# 高分
# import re
# pattern1 = re.compile('[^\d]+')

# 以下列第一种形式即可获得评论的字符串
# print(type(data2[data2['score']>7]['content'][3003]))
# print(pattern1.findall(data2[data2['score']>7]['content'][3003]))

plot_word_cloud_maoyan(data=data2[data2['score']>7], swords=swords, path="Worldcloud_favorate")
# 低分
plot_word_cloud_maoyan(data=data2[data2['score']<4], swords=swords, path="Worldcloud_poor")

In [41]:
# 猫眼高分的部分评论
data2[data2['score']>7]


,content,gender,nick,replyCount,score,time,upCount,userId,userLevel
time,,,,,,,,,
2019-02-11 10:10:00,很好震撼的很啊,1.0,江生,0.0,9.0,2019-02-11 10:10:00,0.0,2.016950e+08,3.0
2019-02-11 10:10:00,国产的科幻巅峰，丝毫不逊色好莱坞科幻大片，影响力就像星球大战一样，是国产科幻的开山之做，希望...,0.0,Decadence,0.0,10.0,2019-02-11 10:10:00,0.0,1.020876e+09,4.0
2019-02-11 10:10:00,超级好看啊 户口帅的一批 我自己一个人看的 看得热血沸腾啊 爷爷死的时候真的哭死了,2.0,暴走萝莉912,0.0,10.0,2019-02-11 10:10:00,0.0,3.042996e+08,3.0
2019-02-11 10:10:00,保护生态环境，从我做,1.0,*^_^*红尘逍遥,0.0,10.0,2019-02-11 10:10:00,0.0,1.548123e+09,2.0
2019-02-11 10:10:00,超级好看，感动，特效好哒,2.0,罗大大,0.0,9.0,2019-02-11 10:10:00,0.0,8.973641e+08,2.0
...,...,...,...,...,...,...,...,...,...
2019-02-11 01:40:00,终于明白什么叫饱和式救援\n人类命运共同体选择了希望\n不管走多远，也要带着哪怕是残破的家\n,1.0,ficeray,0.0,10.0,2019-02-11 01:40:00,0.0,2.455027e+09,1.0
2019-02-11 01:40:00,很不错，推荐大家看。,1.0,愛•菩提•翡翠,0.0,10.0,2019-02-11 01:40:00,0.0,2.297931e+09,1.0
2019-02-11 01:40:00,堪比好莱坞大片，震撼,1.0,Zz,0.0,10.0,2019-02-11 01:40:00,0.0,2.223646e+09,2.0


In [42]:
# 猫眼低分的部分评论
data2[data2['score']<4]

,content,gender,nick,replyCount,score,time,upCount,userId,userLevel
time,,,,,,,,,
2019-02-11 10:08:00,本来是要买初一的飞驰人生，喜剧之王和外星人的。剧组的同事给我买错了买了这个没买飞驰人生。大年...,1.0,Zjnnn,0.0,3.0,2019-02-11 10:08:00,1.0,1.032167e+09,2.0
2019-02-11 10:06:00,差了......,1.0,ReturnO,0.0,1.0,2019-02-11 10:06:00,0.0,1.034489e+09,3.0
2019-02-11 10:03:00,我不可以。。,0.0,贺岁,1.0,1.0,2019-02-11 10:03:00,0.0,1.699866e+09,2.0
2019-02-11 10:02:00,简直就是垃圾电影,1.0,cy。,4.0,1.0,2019-02-11 10:02:00,2.0,1.773611e+09,2.0
2019-02-11 10:02:00,太烂，高分哪里来的,2.0,蓝玫瑰1688888,2.0,2.0,2019-02-11 10:02:00,3.0,2.216922e+08,2.0
...,...,...,...,...,...,...,...,...,...
2019-02-11 01:51:00,看不懂，真的看不懂，很多剧情穿越的太快了,0.0,AoJ835042755,0.0,1.0,2019-02-11 01:51:00,0.0,5.213839e+08,1.0
2019-02-11 01:48:00,电影我还没看，但我就想问问，万达电影票100多一张？VIP厅就这么贵么？还是天价电影票？这些...,1.0,曄,0.0,2.0,2019-02-11 01:48:00,2.0,1.023754e+09,2.0
2019-02-11 01:45:00,孩子来看流浪地球电影是3d的，和你们要眼镜检票员说不是3d，没给~结果孩子就凑合模模糊糊看完...,0.0,yEf246298079,0.0,1.0,2019-02-11 01:45:00,0.0,8.726157e+08,2.0


In [43]:
# # 猫眼高分评论
# 数量过多，需要解除注释即可查看
# for i in data2[data2['score']>7]['content']:
#     print(i+'\n')

In [44]:
# # 猫眼低分评论
# for i in data2[data2['score']<4]['content']:
#     print(i+'\n')

#### 5. 评分的人群有哪些特征？（性别、等级）

##### 1). 评分和性别及等级的关系(都只有在猫眼中才有)

In [45]:
# 评分和性别的关系
# 0代表未知，1代表男，2代表女

# 获得总体各个性别的个数
gender_total_maoyan = data2['gender'].value_counts().values
# print(gender_total_maoyan.values)

# 获得低分各个性别的个数
gender_low_maoyan = data2.loc[data2['score']<4, 'gender'].value_counts().values

# 获得低分各个性别的个数
gender_high_maoyan = data2.loc[data2['score']>7, 'gender'].value_counts().values

# print(data2['gender'].value_counts())

# 将对应性别个数转换为int类型
total_sex = []
for i in gender_total_maoyan:
    total_sex.append(int(i))

# 将低分对应性别个数转换为int类型
low_sex = []
for j in gender_low_maoyan:
    low_sex.append(int(j))
# 数据库中猫眼低分评论只有3条，2条没有说明性别，1条为男性,因此添加女性数为0
# low_sex.append(0)

# 将高分对应性别个数转换为int类型
high_sex = []
for k in gender_high_maoyan:
    high_sex.append(int(k))

sum1 = total_sex[1] + total_sex[2]
sum2 = high_sex[1] + high_sex[2]
sum3 = low_sex[1] + low_sex[2]

# 将未知人数去掉并且求得每一种男女所占比例
total_sex_per = [round(total_sex[1]/sum1 * 100, 2), round(total_sex[2]/sum1 * 100, 2)]
high_sex_per = [round(high_sex[1]/sum2 * 100, 2), round(high_sex[2]/sum2 * 100, 2)]
low_sex_per = [round(low_sex[1]/sum3 * 100, 2), round(low_sex[2]/sum3 * 100, 2)]
# low_per = [low_sex[1]/sum3, low_sex[2]/sum3]
# print(total_sex,high_per,low_sex)


In [46]:
# 评分和等级的关系

# 获得总体各个等级的个数
level_total_maoyan = data2['userLevel'].value_counts().sort_index().values
# print(gender_total_maoyan.values)

# 获得低分各个等级的个数
level_low_maoyan = data2.loc[data2['score']<4, 'userLevel'].value_counts().sort_index().values

# 获得高分各个等级的个数
level_high_maoyan = data2.loc[data2['score']>7, 'userLevel'].value_counts().sort_index().values

# 用于查看用户等级和人数
# print(data2['userLevel'].value_counts().sort_index())
# print(data2.loc[data2['score']<4, 'userLevel'].value_counts().sort_index())
# print(data2.loc[data2['score']>7, 'userLevel'].value_counts().sort_index())
# print(level_high_maoyan,level_low_maoyan,level_total_maoyan)

# 将对应等级个数转换为int类型
total_level = []
for i in level_total_maoyan:
    total_level.append(int(i))

# 将低分对应等级个数转换为int类型
low_level = []
for j in level_low_maoyan:
    low_level.append(int(j))
# 数据库中猫眼低分评论没有1，3，4级的用户,因此令对应等级的人数为0


# 将高分对应等级个数转换为int类型
high_level = []
for k in level_high_maoyan:
    high_level.append(int(k))

sum1 = 0
sum2 = 0
sum3 = 0

# 计算总体，高分，低分不同等级的总人数
for i in range(len(total_level)):
    sum1 += total_level[i]
    sum2 += high_level[i]
    sum3 += low_level[i]

total_level_per = []
high_level_per = []
low_level_per = []

# 求各个等级的所占比例
for i in range(len(total_level)):

    total_level_per.append(round(total_level[i]/sum1 * 100, 2))
    high_level_per.append(round(high_level[i]/sum2 * 100, 2))
    low_level_per.append(round(low_level[i]/sum3 * 100, 2))

# print(total_level_per, high_level_per, low_level_per)


In [47]:
# 绘制《流浪地球》观众性别，等级条形图和饼图

# 绘制性别分布条形图
def sex_distribution_bar() -> Bar:
    bar = (
        Bar()
        .add_xaxis(['未知', '男', '女'])
        .add_yaxis("总体性别分布", total_sex)
        .add_yaxis("高分性别分布", high_sex)
        .add_yaxis("低分性别分布", low_sex)
        .set_global_opts(title_opts=opts.TitleOpts(title="观众性别分布条形图"))
    )
    return bar

# 绘制性别分布饼图
def sex_distribution_pie() -> Pie:
    fn = """
        function(params) {
            if(params.name == '女')
                return '\\n\\n\\n' + params.name + ' : ' + params.value + '%';
            return params.name + ' : ' + params.value + '%';
        }
        """

    def new_label_opts():
        return opts.LabelOpts(formatter=JsCode(fn), position="center")

    pie = (
        Pie()
        .add(
            "",
            [list(z) for z in zip(['男', '女'], total_sex_per)],
            center=["15%", "30%"],
            radius=[60, 80],
            label_opts=new_label_opts(),
        )
        .add(
            "",
            [list(z) for z in zip(['男', '女'], high_sex_per)],
            center=["40%", "30%"],
            radius=[60, 80],
            label_opts=new_label_opts(),
        )
        .add(
            "",
            [list(z) for z in zip(['男', '女'], low_sex_per)],        #没有女性特殊处理
            center=["65%", "30%"],
            radius=[60, 80],
            label_opts=new_label_opts(),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="观众性别分布饼图"),
            legend_opts=opts.LegendOpts(
                type_="scroll", pos_top="20%", pos_left="80%", orient="vertical"
            ),
        )
    )

    return pie

# 查看数据格式
# print([list(z) for z in zip(['男', '女'], low_sex_per)])
# print([list(z) for z in zip(['0', '1', '2', '3', '4', '5', '6'], total_level_per)])

# 绘制等级分布条形图
def level_distribution_bar() -> Bar:
    bar = (
        Bar()
        .add_xaxis([0, 1, 2, 3, 4, 5, 6])
        .add_yaxis("总体等级分布", total_level)
        .add_yaxis("高分等级分布", high_level)
        .add_yaxis("低分等级分布", low_level)
        .set_global_opts(title_opts=opts.TitleOpts(title="观众等级分布条形图"))
    )
    return bar

# 绘制等级分布饼图
def level_distribution_pie() -> Pie:
    fn = """
        function(params) {
            if(params.name == '5')
                return '\\n\\n\\n\\n\\n\\n\\n\\n\\n' + params.name + ' : ' + params.value + '%';
            if(params.name == '4')
                return '\\n\\n\\n\\n\\n\\n\\n' + params.name + ' : ' + params.value + '%';
            if(params.name == '3')
                return '\\n\\n\\n\\n\\n' + params.name + ' : ' + params.value + '%';
            if(params.name == '2')
                return '\\n\\n\\n' + params.name + ' : ' + params.value + '%';
            if(params.name == '1')
                return '\\n' + params.name + ' : ' + params.value + '%';
            if(params.name == '0' || params.name == '6')
                return ' ';

        }
        """

    def new_label_opts():
        return opts.LabelOpts(formatter=JsCode(fn), position="center")

    # 绘制饼图的时候1，2，3，4类型要是字符串类型
    pie = (
        Pie()
        .add(
            "",
            [list(z) for z in zip(['0', '1', '2', '3', '4', '5', '6'], total_level_per)],
            center=["15%", "30%"],
            radius=[80, 100],
            label_opts=new_label_opts(),
        )
        .add(
            "",
            [list(z) for z in zip(['0', '1', '2', '3', '4', '5', '6'], high_level_per)],
            center=["40%", "30%"],
            radius=[80, 100],
            label_opts=new_label_opts(),
        )
        .add(
            "",
            [list(z) for z in zip(['0', '1', '2', '3', '4', '5', '6'], low_level_per)],        #没有女性特殊处理
            center=["65%", "30%"],
            radius=[80, 100],
            label_opts=new_label_opts(),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="观众等级分布饼图"),
            legend_opts=opts.LegendOpts(
                type_="scroll", pos_top="20%", pos_left="80%", orient="vertical"
            ),
        )
    )

    return pie


# 将条形图和饼图放在一起展示
def page_layout1() -> Page:
    page = Page(layout=Page.SimplePageLayout)
    page.add(
        sex_distribution_bar(),
        sex_distribution_pie()

    )
    page.render("./Charts_html/Sex_distribution.html")
    return  page


def page_layout2() -> Page:
    page = Page(layout=Page.SimplePageLayout)
    page.add(
        level_distribution_bar(),
        level_distribution_pie()

    )
    page.render("./Charts_html/Level_distribution.html")
    return  page



if __name__ == "__main__":
    page_layout1()
    page_layout2()

# tab = Tab()
# tab.add(page_layout1(), "《流浪地球》观众性别分布")
# tab.add(page_layout2(), "《流浪地球》观众等级分布")
# tab.add(sex_distribution_bar(), "《流浪地球》观众性别分布条形图")
# tab.add(sex_distribution_pie(), "《流浪地球》观众性别分布饼图")
# tab.add(level_distribution_bar(), "《流浪地球》观众等级分布条形图")
# tab.add(level_distribution_pie(), "《流浪地球》观众等级分布饼图")

# tab.render("./charts_html/Sex_Level_distribution.html")

##### 6. 高低分跟哪位演员有关？

In [48]:
# 将演员真名，真名绰号，角色绰号和电影角色名做一个映射
mapping = {'liucixin':'刘慈欣|大刘', 'guofan':'郭帆', 'quchuxiao':'屈楚萧|刘启|户口|男主', 'wujing':'吴京|刘培强|战狼',
           'liguangjie':'李光洁|王磊', 'wumengda':'吴孟达|达叔|韩子昂', 'zhaojinmai':'赵今麦|韩朵朵|女主'}

# 将演员名字的拼音作为数据的新字段，若该条记录出现了该演员，则将其名字的拼音对应为true
for key, value in mapping.items():
    data2[key] = data2['content'].str.contains(value)

In [49]:
# 节约栈空间
# data2.info

### 总体提及次数

In [50]:
# 将对应的人名拼音和在数据中的出现次数形成字典

# 因为数据存在Nah值，无法处理，因此pd.Series({key: data.loc[data[key], 'score']无法使用，但是使用分数≥0也可以做到相同效果
staff_count = pd.Series({key: data2.loc[data2[key]&(data2['score']>=0), 'score'].count() for key in mapping.keys()}).sort_index()
staff_count

guofan         272
liguangjie     171
liucixin      3122
quchuxiao     1274
wujing        5738
wumengda       477
zhaojinmai     761
dtype: int64

In [51]:
# 求低分评论中出现的演员名及其个数
staff_count_low = pd.Series({key: data2.loc[data2[key]&(data2['score']<4), 'score'].count() for key in mapping.keys()}).sort_index()
staff_count_low

guofan          3
liguangjie      4
liucixin       28
quchuxiao      65
wujing        309
wumengda       26
zhaojinmai     40
dtype: int64

In [52]:
# 求高分评论中出现的演员名及其个数
staff_count_high = pd.Series({key: data2.loc[data2[key]&(data2['score']>7), 'score'].count() for key in mapping.keys()}).sort_index()
staff_count_high

guofan         267
liguangjie     153
liucixin      2975
quchuxiao     1055
wujing        4984
wumengda       404
zhaojinmai     602
dtype: int64

In [53]:
# 演员在低分评论的提及次数 占 所有评论中该演员的提及次数
staff_count_pct1 = np.round(staff_count_low/staff_count*100, 2).sort_index()
staff_count_pct1

guofan        1.10
liguangjie    2.34
liucixin      0.90
quchuxiao     5.10
wujing        5.39
wumengda      5.45
zhaojinmai    5.26
dtype: float64

In [54]:
# 演员在高分评论的提及次数 占 所有评论中该演员的提及次数
staff_count_pct2 = np.round(staff_count_high/staff_count*100, 2).sort_index()
staff_count_pct2

guofan        98.16
liguangjie    89.47
liucixin      95.29
quchuxiao     82.81
wujing        86.86
wumengda      84.70
zhaojinmai    79.11
dtype: float64

In [55]:
# 绘制演员在低分/高分评论的提及次数 占 所有评论中该演员的提及次数的柱状图

from pyecharts.globals import ThemeType
# 所有演员以下列顺序出现
# ['guofan', 'liguangjie', 'liucixin', 'quchuxiao', 'wujing', 'wumengda','zhaojinmai']

# 存放演员的总出现次数
count1 = []
# 存放好评中演员的总出现次数
count2 = []
# 存放差评中演员的总出现次数
count3 = []

for i in range(len(staff_count)):
    count1.append(int(staff_count[i]))
    count2.append(int(staff_count_high[i]))
    count3.append(int(staff_count_low[i]))

# print(count1,count2,count3)
list1 = []
list2 = []
list3 = []

# 将数据转换为指定的数据格式
'''
    [
    {"value": 12, "percent": 12 / (12 + 3)},
    {"value": 23, "percent": 23 / (23 + 21)},
    ...
    ]
'''

for i in range(len(staff_count)):
    temp1 = {}
    temp2 = {}
    temp3 = {}

    temp1["value"] = count2[i]
    temp1["percent"] = count2[i] / count1[i]
    list1.append(temp1)
    temp2["value"] = count3[i]
    temp2["percent"] = count3[i] / count1[i]
    list2.append(temp2)
    temp3["value"] = (count1[i] - count2[i] - count3[i])
    temp3["percent"] = (count1[i] - count2[i] - count3[i]) / count1[i]
    list3.append(temp3)


#### 低分评论所涉及到的主要演员名字

##### 1.吴孟达

In [56]:
data2[data2['wumengda']&(data2['score']<4)].nlargest(5, 'upCount')

,content,gender,nick,replyCount,score,time,upCount,userId,userLevel,liucixin,guofan,quchuxiao,wujing,liguangjie,wumengda,zhaojinmai
time,,,,,,,,,,,,,,,,
2019-02-05 14:11:00,可惜了60+大洋，达叔的演技是唯一的亮点，小鲜肉们稚嫩且乏味的演技（约等于0.1）好无聊，和...,1.0,熊晓军［德恒律师］,13.0,1.0,2019-02-05 14:11:00,42.0,1.154499e+09,5.0,False,False,False,False,False,True,False
2019-02-11 02:14:00,真的，这一分我都是给的吴孟达和雷佳音的，剩下的真的不能多给了，电影，最基本的是故事情节，一味...,2.0,林艾G,52.0,1.0,2019-02-11 02:14:00,40.0,6.345564e+07,3.0,False,False,False,False,False,True,False
2019-02-08 04:46:00,主人公太扯了，三观不正，净瞎拍，拿着吴京吴孟达做幌子，真浪费我一百大洋。这片子也就骗骗小孩能...,1.0,博爱永恒,11.0,3.0,2019-02-08 04:46:00,39.0,5.308690e+07,2.0,False,False,False,True,False,True,False
2019-02-09 04:33:00,也别给半星了，剧情太烂，情节不好，故事不合理，尤其最后爆炸，还能爆炸5000公里，飞船都让你...,1.0,闹闹先生丶丶,18.0,2.0,2019-02-09 04:33:00,31.0,1.737550e+08,3.0,False,False,False,True,False,True,False
2019-02-07 10:19:00,台词设定太尴尬，强行煽情，那个外国人设定最恶心，强行设定的人物，吴京，吴孟达只是客串，还有那...,0.0,A_啊杰,7.0,1.0,2019-02-07 10:19:00,28.0,1.033874e+09,4.0,False,False,False,True,False,True,False


In [57]:
for i in data2[data2['wumengda']&(data2['score']<4)].nlargest(5, 'upCount')['content']:
    print(i+'\n')

可惜了60+大洋，达叔的演技是唯一的亮点，小鲜肉们稚嫩且乏味的演技（约等于0.1）好无聊，和看昨晚的春晚感觉差不多，好像喊得很大声，哭得很撕心，但就是没内容，演得太空洞。催泪不够真诚，搞笑不够认真，细节太过毛糙，整个就是一部被制作，为制作而制作的视听片，片尾曲让人无力吐槽。一个人的初一良辰就这么被拱了。

真的，这一分我都是给的吴孟达和雷佳音的，剩下的真的不能多给了，电影，最基本的是故事情节，一味强调特技多么好多么好，去看科技展不就好了？真的是浪费了80块的电影票，毫无情节可言，一味地个人英雄主义，强行煽情，把观众得智商按在地上摩擦，这样的烂片是买了水军吧……垃圾！

主人公太扯了，三观不正，净瞎拍，拿着吴京吴孟达做幌子，真浪费我一百大洋。这片子也就骗骗小孩能行，真的不敢恭维。

也别给半星了，剧情太烂，情节不好，故事不合理，尤其最后爆炸，还能爆炸5000公里，飞船都让你开这么快了，还用的着点燃么？直接开进木星不就好了，飞船后面喷的不是火啊？送达叔一张票，卖战狼一份情，这电影真不行

台词设定太尴尬，强行煽情，那个外国人设定最恶心，强行设定的人物，吴京，吴孟达只是客串，还有那人物设定，太牵强，不知道9.3分怎么来的



##### 2.吴京

In [58]:
data2[data2['wujing']&(data2['score']<4)].nlargest(5, 'upCount')

,content,gender,nick,replyCount,score,time,upCount,userId,userLevel,liucixin,guofan,quchuxiao,wujing,liguangjie,wumengda,zhaojinmai
time,,,,,,,,,,,,,,,,
2019-02-07 23:00:00,太多人给这部片高分了，我必须给低分把这部电影的评分拉低，因为他不值得9点多分，顶多值6.5分...,1.0,杨某X,42.0,2.0,2019-02-07 23:00:00,95.0,8.550003e+07,1.0,False,False,False,True,False,False,False
2019-02-06 17:21:00,逻辑非常不严密，既然定位是科幻片，起码的逻辑得有吧？\n1.一个0.7级宇宙文明等级的星球，...,1.0,团拼丶小王子,108.0,1.0,2019-02-06 17:21:00,58.0,8.675410e+06,3.0,False,False,False,True,False,False,False
2019-02-07 03:22:00,开头以为是国产科幻大片，气象不凡，道具方面下了不少功夫，某些场景直追好莱坞。整体上离好莱坞还...,1.0,meituanchzhh,33.0,1.0,2019-02-07 03:22:00,49.0,3.405231e+07,3.0,False,False,False,True,False,False,False
2019-02-05 09:17:00,什么玩意儿，发动机和石头砸下来的声音从头震到尾，演技拙劣的儿女情长的片段令人作呕，吴京一副大...,0.0,westbrookFMVP,36.0,1.0,2019-02-05 09:17:00,48.0,7.795206e+08,2.0,False,False,False,True,False,False,False
2019-02-08 03:18:00,打不了零分，真可惜，一分给吴京吧。烂！,1.0,椰风挡不住.,26.0,1.0,2019-02-08 03:18:00,48.0,1.109243e+09,3.0,False,False,False,True,False,False,False


In [59]:
for i in data2[data2['wujing']&(data2['score']<4)].nlargest(5, 'upCount')['content']:
    print(i+'\n')

太多人给这部片高分了，我必须给低分把这部电影的评分拉低，因为他不值得9点多分，顶多值6.5分样子，在国产片里面算不错的，剧情烂，东拉西扯的，地球你造发动机推动脱离太阳引力我也忍了，点燃木星就太扯了，点燃木星瞬间推动地球逃脱木星引力，那么大的冲击力地球生物不死绝鬼都不信！还有作为灾难科幻片，震撼人心的灾难场景基本没有，地球停海啸爆发，我一直在期待灾难爆发时刻人类逃亡的场景，但是从头到尾都没有！走到哪里都挂着国旗，又是打爱国情怀，很符合吴京蹭爱国热度的习惯，反正不会错，但是我就觉得很作！恶心！

逻辑非常不严密，既然定位是科幻片，起码的逻辑得有吧？
1.一个0.7级宇宙文明等级的星球，自己这块地的资源分配都没整明白，是哪来的能量去推着地球改变行星轨迹？
2.点燃木星这脑洞是有多大？几十亿年有多少陨石彗星冲击都没点燃，你突发奇想放个火就能点燃了？全是氢气，是怎么点燃的？就这种气体混合比，怎么点燃的？麻烦把化学反应方程式告诉我一下。还不如说丢个炮仗上去炸一下，起码炮仗还能在无氧的环境下燃烧爆炸。
3.就吴京这种个人英雄主义的演员拿来演这种格局庞大的科幻电影合适吗？单说人物设定，就这种人的军衔是怎么给的？是怎么选为航天员的？有没有做过心理测试？有没有做过性格评估？人类的命运最后是堵出来的？

开头以为是国产科幻大片，气象不凡，道具方面下了不少功夫，某些场景直追好莱坞。整体上离好莱坞还是差半个世纪。1⃣️演员。格式化表演，看不出“人”性内容，几处人物牺牲逻辑性和共鸣性不强，尤其吴京最后牺牲前一段烈士般台词，沿习战狼做派，略显矫情。2⃣️布景。感觉一直都是漫天飞雪、错杂纷乱，缺乏变化。整体风格压抑，没有情景沉入感、带入感。

什么玩意儿，发动机和石头砸下来的声音从头震到尾，演技拙劣的儿女情长的片段令人作呕，吴京一副大义凛然的样子太过难受，什么中国第一部科幻，情节稀烂，特效主要靠音效，加上刻意塑造的中国式英雄，一分没得说

打不了零分，真可惜，一分给吴京吧。烂！



##### 3.赵金麦（女主）

In [60]:
data2[data2['zhaojinmai']&(data2['score']<5)].nlargest(5, 'upCount')

,content,gender,nick,replyCount,score,time,upCount,userId,userLevel,liucixin,guofan,quchuxiao,wujing,liguangjie,wumengda,zhaojinmai
time,,,,,,,,,,,,,,,,
2019-02-08 04:14:00,电影就是电影不要嘈杂着其他因素，就电影本质来说，这电影特效，大场面，做的让我赞叹不已，非常的...,1.0,一抹晨曦784,21.0,3.0,2019-02-08 04:14:00,75.0,1.564863e+08,2.0,False,False,True,False,False,False,True
2019-02-07 03:59:00,就这样吧，特效还行，科幻部分不做评价，就人物刻画来讲已经槽点满满了！刘启就是个狂躁症患者，什...,2.0,CpJ26143934,35.0,4.0,2019-02-07 03:59:00,53.0,9.157036e+07,2.0,False,False,True,True,False,False,True
2019-02-06 15:46:00,刘慈欣《流浪地球》据说是硬科幻，设定感觉可以出一个系列，可导演和编剧讲故事的能力都差强人意，...,2.0,pSU598729902,23.0,1.0,2019-02-06 15:46:00,35.0,1.054238e+09,3.0,True,False,True,False,False,False,True
2019-02-06 10:52:00,3分给特效和创意，男女主真是全程让人吐槽无语\n只是说了一下男女主让人无语，就被追着说，那就...,0.0,夏天爱养猪,12.0,1.0,2019-02-06 10:52:00,35.0,7.095032e+07,2.0,False,False,False,False,False,False,True
2019-02-08 18:10:00,感觉好多角色都是多余的，尤其那两个戏份最多的男女主角,0.0,心在流浪,8.0,2.0,2019-02-08 18:10:00,35.0,1.019022e+09,3.0,False,False,False,False,False,False,True


In [61]:
for i in data2[data2['zhaojinmai']&(data2['score']<5)].nlargest(5, 'upCount')['content']:
    print(i+'\n')


电影就是电影不要嘈杂着其他因素，就电影本质来说，这电影特效，大场面，做的让我赞叹不已，非常的有气势，但是演员演技实在捉急，特别是两个主角，男主人设叛逆性格太过了，女主那段演讲是本片的最大败笔，看完那段演讲让我尴尬的有种跑出电影院的冲动，每个人物死掉都会莫名其妙的来段演讲煽情戏，编剧真的是尬王！总的来说，网上给予这片子得评分没有很客观，相比较国外科幻大片，流浪地球有进步，但是还差得远！电影就是电影跟爱不爱国无关，请不要道德绑架！

就这样吧，特效还行，科幻部分不做评价，就人物刻画来讲已经槽点满满了！刘启就是个狂躁症患者，什么都是别人的错，对着姥爷一口一个老东西，姥爷临死也没改口；韩朵朵就是一**少女，紧要关头还声情并茂朗读小作文；刘培强是我要救儿子……以为儿子死了：好，我放弃了，我听moss的回去休眠……哦！儿子没死，太好了，我要想办法，一定要救地球……🙄️🙄️🙄️

刘慈欣《流浪地球》据说是硬科幻，设定感觉可以出一个系列，可导演和编剧讲故事的能力都差强人意，镜头语言幼稚，几乎全程在快速地晃荡，人物情感没有张力，演员功力都弱弱的，都末日了，还不咸不淡的语气。。男主对他爹的大恨是从哪儿来的？？不就是母亲放弃治疗吗？那个韩朵朵出来是干什么的？是让大家保护的“希望”化身？？强行煽情的台词让我汗毛竖起。刻意卖的末世日情怀，我不想接。狂热的拥护。

原来酒带上太空舱是为了炸掉智能机器人莫斯的。。被酒燃烧前莫斯的话是经典——想让人类保持理智是一种奢望。笑岔我。

看到东方明珠塔被雪冻住了我就忍不住笑，不知道老美看到纽约被炸那么多次什么心情。

无知导致浪漫。



3分给特效和创意，男女主真是全程让人吐槽无语
只是说了一下男女主让人无语，就被追着说，那就改评一分了

感觉好多角色都是多余的，尤其那两个戏份最多的男女主角



##### 4.屈楚萧（男主）

In [62]:
data2[data2['quchuxiao']&(data2['score']<5)].nlargest(5, 'upCount')

,content,gender,nick,replyCount,score,time,upCount,userId,userLevel,liucixin,guofan,quchuxiao,wujing,liguangjie,wumengda,zhaojinmai
time,,,,,,,,,,,,,,,,
2019-02-08 04:14:00,电影就是电影不要嘈杂着其他因素，就电影本质来说，这电影特效，大场面，做的让我赞叹不已，非常的...,1.0,一抹晨曦784,21.0,3.0,2019-02-08 04:14:00,75.0,1.564863e+08,2.0,False,False,True,False,False,False,True
2019-02-07 03:59:00,就这样吧，特效还行，科幻部分不做评价，就人物刻画来讲已经槽点满满了！刘启就是个狂躁症患者，什...,2.0,CpJ26143934,35.0,4.0,2019-02-07 03:59:00,53.0,9.157036e+07,2.0,False,False,True,True,False,False,True
2019-02-06 08:27:00,剧情相当尬，男主属于看着就让人不舒服的那种。这网评真的不是水军刷出来的吗。。。,0.0,lu,15.0,4.0,2019-02-06 08:27:00,44.0,1.241587e+09,4.0,False,False,True,False,False,False,False
2019-02-08 08:59:00,看得出导演用心了，特效值得肯定，还有保护地球热爱家园的主旨定位没问题，但是，演员的演技不敢恭...,1.0,沧哥,7.0,3.0,2019-02-08 08:59:00,37.0,2.997020e+08,3.0,False,False,True,False,False,False,False
2019-02-06 15:46:00,刘慈欣《流浪地球》据说是硬科幻，设定感觉可以出一个系列，可导演和编剧讲故事的能力都差强人意，...,2.0,pSU598729902,23.0,1.0,2019-02-06 15:46:00,35.0,1.054238e+09,3.0,True,False,True,False,False,False,True


In [63]:
for i in data2[data2['quchuxiao']&(data2['score']<5)].nlargest(5, 'upCount')['content']:
    print(i+'\n')

电影就是电影不要嘈杂着其他因素，就电影本质来说，这电影特效，大场面，做的让我赞叹不已，非常的有气势，但是演员演技实在捉急，特别是两个主角，男主人设叛逆性格太过了，女主那段演讲是本片的最大败笔，看完那段演讲让我尴尬的有种跑出电影院的冲动，每个人物死掉都会莫名其妙的来段演讲煽情戏，编剧真的是尬王！总的来说，网上给予这片子得评分没有很客观，相比较国外科幻大片，流浪地球有进步，但是还差得远！电影就是电影跟爱不爱国无关，请不要道德绑架！

就这样吧，特效还行，科幻部分不做评价，就人物刻画来讲已经槽点满满了！刘启就是个狂躁症患者，什么都是别人的错，对着姥爷一口一个老东西，姥爷临死也没改口；韩朵朵就是一**少女，紧要关头还声情并茂朗读小作文；刘培强是我要救儿子……以为儿子死了：好，我放弃了，我听moss的回去休眠……哦！儿子没死，太好了，我要想办法，一定要救地球……🙄️🙄️🙄️

剧情相当尬，男主属于看着就让人不舒服的那种。这网评真的不是水军刷出来的吗。。。

看得出导演用心了，特效值得肯定，还有保护地球热爱家园的主旨定位没问题，但是，演员的演技不敢恭维，确实尴尬，台词尴尬，表情尴尬，男主角给人一种混二愣熊孩子的感觉，没有人格定位。剧情拖沓，逻辑生硬，强行植入煽情片段，明明是义无反顾的离家出走又一个个把家说的比什么都重要！作为国产科幻片来说可圈可点，值得肯定，应该给八分鼓励，但我看了这里的评论发现，但凡有说哪里不好的，都会被打成美帝，被说成无脑，我就尴尬了，我和那些开口就骂人的人肯定不是一路，评分要是也跟他们一样，我也会被误认为是那些人吧？所以，我给三分！

刘慈欣《流浪地球》据说是硬科幻，设定感觉可以出一个系列，可导演和编剧讲故事的能力都差强人意，镜头语言幼稚，几乎全程在快速地晃荡，人物情感没有张力，演员功力都弱弱的，都末日了，还不咸不淡的语气。。男主对他爹的大恨是从哪儿来的？？不就是母亲放弃治疗吗？那个韩朵朵出来是干什么的？是让大家保护的“希望”化身？？强行煽情的台词让我汗毛竖起。刻意卖的末世日情怀，我不想接。狂热的拥护。

原来酒带上太空舱是为了炸掉智能机器人莫斯的。。被酒燃烧前莫斯的话是经典——想让人类保持理智是一种奢望。笑岔我。

看到东方明珠塔被雪冻住了我就忍不住笑，不知道老美看到纽约被炸那么多次什么心情。

无知导致浪漫。





#### 高分评论所涉及到的主要演员名字

##### 1.吴孟达

In [64]:
data2[data2['wumengda']&(data2['score']>7)].nlargest(5, 'upCount')

,content,gender,nick,replyCount,score,time,upCount,userId,userLevel,liucixin,guofan,quchuxiao,wujing,liguangjie,wumengda,zhaojinmai
time,,,,,,,,,,,,,,,,
2019-02-05 01:03:00,外骨骼找的世界上最好的外骨骼制作团队，一套外骨骼50万元。音乐录制去的世界上最好的录音棚《艾...,0.0,hKP853271285,1.0,10.0,2019-02-05 01:03:00,20.0,8.066066e+08,1.0,False,True,False,False,False,True,False
2019-02-09 07:20:00,之前已有朋友看过，对之评价很高，因此非常期待。\n\n果然名副其实，大大超过预期，国产科幻片...,0.0,格林纯姐姐,1.0,10.0,2019-02-09 07:20:00,15.0,9.584794e+07,3.0,False,False,False,True,False,True,False
2019-02-06 18:24:00,不知道，大家都有没有注意到达叔的一句话，“在我们那个年代大家都不太关心太阳，只关心一样东西，...,1.0,Mr·6先生,1.0,10.0,2019-02-06 18:24:00,13.0,6.589161e+08,4.0,False,False,False,False,False,True,False
2019-02-11 05:18:00,1.电影我看了，吴京炸了，京黑们快来买票参观\n2电影里年代是2075年，爷爷韩子昂是99年...,1.0,成达门业李钰霖,3.0,10.0,2019-02-11 05:18:00,9.0,1.653117e+09,2.0,False,False,True,True,False,True,False
2019-02-06 19:25:00,非常棒，科幻灾难大片的既视感，国产科幻进步非常大，构思几近满分，特效也属上乘，男主演的挺招人...,1.0,VKR802692810,0.0,10.0,2019-02-06 19:25:00,9.0,3.821338e+08,3.0,False,False,True,True,False,True,True


In [65]:
for i in data2[data2['wumengda']&(data2['score']>7)].nlargest(5, 'upCount')['content']:
    print(i+'\n')


外骨骼找的世界上最好的外骨骼制作团队，一套外骨骼50万元。音乐录制去的世界上最好的录音棚《艾比路》。总投资保底超过7千万美元，演员没有一个流量明星，吴孟达老师带病拍戏，剧组还有随身医生，每拍完一段都要吸氧。
大部分的资金都用在了特效上，就是为了保证质量。
在不到30天上映的情况下，郭帆自己的工作室再追加900万投资，谁知道这钱哪来的，他可能是押上全部身家了，怕是把房子抵押出去了。
你们应该知道一部投资六七千万美元的“中国”“科幻”电影，没有多少宣发，没有流量明星，意味着多大风险。
几乎必死。
所以说现在很多人还是张口就来，尽管不是喷也不是黑。

之前已有朋友看过，对之评价很高，因此非常期待。

果然名副其实，大大超过预期，国产科幻片中毫无争议的最佳作品，跟国外影片相比也毫不逊色。科幻脑洞开得很大，各种特效非常震撼，故事让人感到很真实，虽是2D观影者仍然感觉身临其境。
情感线也很感人，真实自然，触动人心。“虽千万人，吾往矣”。宇航员和救援人员勇于牺牲的精神让人联想起王伟和我们的航天英雄，小女孩播报那一段更是看得笔者潸然泪下。
唯一的不足是吴京和吴孟达太熟悉了让我老跳戏，哥哥的表演也有些稚嫩。
春节档的影片敢提前大规模点映，现在看来，主创人员的自信是非常有道理的。
值得强烈推荐，春节期间准备二刷。

不知道，大家都有没有注意到达叔的一句话，“在我们那个年代大家都不太关心太阳，只关心一样东西，钱。”其实这种观点，是值得思考的。抛开科幻作品的商业价值，这种社会价值可以用一句话体现在在科幻作品里，并且直击心灵其实就是最大一种讽刺。我们现在的生活，不管大家对社会秉持什么样的观点，不可否认其实是前人用鲜血用生命用疾苦换来的，当一个以命为生存前提的时代结束难道就要以钱来代替继续前行么？以钱换物质，那么真的没精神什么事儿了！人类最伟大的是爱，也如同本片人工智能最后的那句话。所以劝解诸君，多多自律多多勤俭。切莫只认钱不屑情啊！

1.电影我看了，吴京炸了，京黑们快来买票参观
2电影里年代是2075年，爷爷韩子昂是99年生的所以他会玩手机跳海草海草。
3.引爆木星以色列科学家已经想到，但保住空间站，延续人类文明才是最佳选择，因此说成功率为零。
4.其实主角一行人没有成功过，连最后的引爆木星都是刘培强牺牲自己和空间站引爆的。
5.电影里有地面叛军与李一一的救援队枪战的情节，被剪了，所以救援

##### 2.吴京

In [66]:
data2[data2['wujing']&(data2['score']>7)].nlargest(5, 'upCount')

,content,gender,nick,replyCount,score,time,upCount,userId,userLevel,liucixin,guofan,quchuxiao,wujing,liguangjie,wumengda,zhaojinmai
time,,,,,,,,,,,,,,,,
2019-02-05 14:44:00,前部分虽有不足，但特效是超出预期，后劲较足，到了中后部分有两个地方感动到我，刘培强实现预言，...,2.0,JuliaFanl,6.0,8.0,2019-02-05 14:44:00,172.0,2.273608e+08,5.0,False,False,True,True,True,False,False
2019-02-05 14:58:00,看完预告就给满分的、我应该是第一个、不管好不好看、这部片子必须支持、因为这是中国电影工业的一...,0.0,明亮,0.0,10.0,2019-02-05 14:58:00,22.0,1.038797e+09,4.0,False,False,False,True,False,False,False
2019-02-05 10:06:00,特效团队已经确定了...是More...特效做的还不错...More做过悟空传，特效应该差不...,1.0,。。。,0.0,10.0,2019-02-05 10:06:00,15.0,1.031318e+09,3.0,False,False,False,True,False,False,False
2019-02-09 07:20:00,之前已有朋友看过，对之评价很高，因此非常期待。\n\n果然名副其实，大大超过预期，国产科幻片...,0.0,格林纯姐姐,1.0,10.0,2019-02-09 07:20:00,15.0,9.584794e+07,3.0,False,False,False,True,False,True,False
2019-02-05 18:39:00,我就莫名其妙了，怎么中国人自己出个什么你们都否认，这部片子怎么就不好了，中国人自己拍的科幻起...,0.0,V悔Z,4.0,10.0,2019-02-05 18:39:00,12.0,4.974724e+08,3.0,False,False,False,True,False,False,False


In [67]:
for i in data2[data2['wujing']&(data2['score']>7)].nlargest(5, 'upCount')['content']:
    print(i+'\n')

前部分虽有不足，但特效是超出预期，后劲较足，到了中后部分有两个地方感动到我，刘培强实现预言，其次是王磊队长说出命令的瞬间。希望是什么？在绝境中不放弃任何一种可能，与其等待死亡，不如拼死一搏！长大又是什么？与家人共同经历之后理解彼此，打开心结。这次拯救地球集结了大家的力量，也见证了刘启的成长。流浪地球计划，让地球不再流浪。

看完预告就给满分的、我应该是第一个、不管好不好看、这部片子必须支持、因为这是中国电影工业的一次大胆尝试和革命。你拍多少部战狼或者多少部优秀的喜剧电影、欧美丝毫不会侧目一下、但是如果科幻题材成功了，不但能带动中国电影产业升级和走出去、更能起到传播中国文化塑造中国英雄的伟大使命……
本人不道德绑架、但是极力推荐......
中国有史以来第一部硬科幻，电审故字2019001号可真不是开玩笑的，不论结果如何、此片已经青史留名了。

特效团队已经确定了...是More...特效做的还不错...More做过悟空传，特效应该差不了...剧组的确是和维塔谈过...不过也就仅限坐下来谈谈了...好像维塔太贵..貌似是按帧收费...剧组预算不够...这个定档日期很神奇...看来是冲着挣钱...有中影投资，还有吴京出演，感觉少不了主旋律...希望故事能讲好... 不过中国第一部硬科幻，挺了！！！！

之前已有朋友看过，对之评价很高，因此非常期待。

果然名副其实，大大超过预期，国产科幻片中毫无争议的最佳作品，跟国外影片相比也毫不逊色。科幻脑洞开得很大，各种特效非常震撼，故事让人感到很真实，虽是2D观影者仍然感觉身临其境。
情感线也很感人，真实自然，触动人心。“虽千万人，吾往矣”。宇航员和救援人员勇于牺牲的精神让人联想起王伟和我们的航天英雄，小女孩播报那一段更是看得笔者潸然泪下。
唯一的不足是吴京和吴孟达太熟悉了让我老跳戏，哥哥的表演也有些稚嫩。
春节档的影片敢提前大规模点映，现在看来，主创人员的自信是非常有道理的。
值得强烈推荐，春节期间准备二刷。

我就莫名其妙了，怎么中国人自己出个什么你们都否认，这部片子怎么就不好了，中国人自己拍的科幻起步。是！他是有瑕疵，但是你不能不承认他用心了，完全碾压那些小鲜肉的五毛钱特效。难道中国人自己用心做出来的东西就那么难被自己人承认吗？拍个《战狼》你们说“个人主义”太强，拍个《红海行动》你们说“太假，就几个人打几百人”和抗战神剧一样

##### 3.赵金麦（女主）

In [68]:
data2[data2['zhaojinmai']&(data2['score']>7)].nlargest(5, 'upCount')

,content,gender,nick,replyCount,score,time,upCount,userId,userLevel,liucixin,guofan,quchuxiao,wujing,liguangjie,wumengda,zhaojinmai
time,,,,,,,,,,,,,,,,
2019-02-05 00:11:00,世界观宏大，特效过关，节奏紧凑。\n就是剧情略有欠缺俗套，改编的不能说很成功，从人类大局观来...,1.0,BrotherDongV5,0.0,9.0,2019-02-05 00:11:00,15.0,152528991.0,2.0,True,False,False,False,False,False,True
2019-02-05 01:54:00,猪年零点首映场观影值了，看完有点打鸡血式兴奋！本片开启中国科幻片里程碑小马达，整体节奏不错，...,0.0,TangY,0.0,8.0,2019-02-05 01:54:00,12.0,265060493.0,3.0,False,False,False,False,False,False,True
2019-02-06 19:25:00,非常棒，科幻灾难大片的既视感，国产科幻进步非常大，构思几近满分，特效也属上乘，男主演的挺招人...,1.0,VKR802692810,0.0,10.0,2019-02-06 19:25:00,9.0,382133795.0,3.0,False,False,True,True,False,True,True
2019-02-08 22:39:00,原来流浪地球还有145分钟和160分钟的版本，其实里面有很多挺重要的部分呀\n比如王磊的妻子...,2.0,fyangle,1.0,10.0,2019-02-08 22:39:00,7.0,85485074.0,2.0,False,False,True,False,True,False,True
2019-02-10 17:52:00,这是我24年人生看过特效最真实最震撼的国产科幻电影。\n唯一扣分项就是女主韩朵朵，我感觉这个...,0.0,Paper灬Tiger丶,0.0,9.0,2019-02-10 17:52:00,6.0,144559792.0,1.0,False,False,False,True,False,False,True


In [69]:
for i in data2[data2['zhaojinmai']&(data2['score']>7)].nlargest(5, 'upCount')['content']:
    print(i+'\n')


世界观宏大，特效过关，节奏紧凑。
就是剧情略有欠缺俗套，改编的不能说很成功，从人类大局观来说，剧情的改编多了太多感性，人类在危机纪元不应该如此感性，不太理智，埋没了大刘对于全人类共同对抗危机时自我牺牲的精神，这点扣分。
男女主演技还是略年轻。

整体感觉还不错，推荐

猪年零点首映场观影值了，看完有点打鸡血式兴奋！本片开启中国科幻片里程碑小马达，整体节奏不错，特效和音乐很棒，片尾曲好听，女主有点傻傻萌，她的煽情台词稍显啰嗦，还有几帧场景宣誓式的台词是硬伤，故事情节大众，总体fine还不赖的一部作品能优点盖过缺点，预测猪年贺岁档黑马一匹！看这类特效影片就得必选IMAX才能完爆体验。

非常棒，科幻灾难大片的既视感，国产科幻进步非常大，构思几近满分，特效也属上乘，男主演的挺招人烦，女主镜头不多但很漂亮，吴京达叔演技还是杠杠滴，非要给意见，那就是如果把木星波推动地球那一瞬间给个镜头绝对震撼。

原来流浪地球还有145分钟和160分钟的版本，其实里面有很多挺重要的部分呀
比如王磊的妻子孩子其实就住在杭州地下城，所以他才要拼了命都要去杭州救他们
韩朵朵出去其实是为了找自己亲生父母的尸体，然后姥爷死了之后韩朵朵和刘启找到了朵朵被救的地方，然后有个特效就是她跪下，然后用手把冰面擦开，冰下全是向上游的尸体。
导演我只想说，我想看160分钟完整版！！！

这是我24年人生看过特效最真实最震撼的国产科幻电影。
唯一扣分项就是女主韩朵朵，我感觉这个角色太多余了，只会大喊大叫。没有动力甲去退“撞针”还占一个使力的位置。特别广播那一段特别尴尬，干嘛不让联合政府来广播？让一个小女孩来，还讲一段故事煽情一下😅有些老套，木星撞地球啊可是亿万火急的呀。 
PS:奥观海同志在哪啊？没看见他🤣。
PS2：吴京撞木星上先把冬眠的生活仓环带脱离了，并不是带着一帮宇航员一起死。  
PS3:地面还可能有幸存者。



##### 4.屈楚萧（男主）

In [70]:
data2[data2['quchuxiao']&(data2['score']>7)].nlargest(5, 'upCount')

,content,gender,nick,replyCount,score,time,upCount,userId,userLevel,liucixin,guofan,quchuxiao,wujing,liguangjie,wumengda,zhaojinmai
time,,,,,,,,,,,,,,,,
2019-02-05 14:44:00,前部分虽有不足，但特效是超出预期，后劲较足，到了中后部分有两个地方感动到我，刘培强实现预言，...,2.0,JuliaFanl,6.0,8.0,2019-02-05 14:44:00,172.0,2.273608e+08,5.0,False,False,True,True,True,False,False
2019-02-08 00:21:00,电影删掉的部分内容:\n1.妹妹想去地面，是因为想找父母的尸体，不是影片里那么中二的理由（所...,0.0,🔥 👦 👔 👖,0.0,9.0,2019-02-08 00:21:00,13.0,1.024416e+09,2.0,False,False,True,False,True,False,False
2019-02-07 02:25:00,“所谓常识不可逆转的时刻，只不过是人类给予自己选择不拼尽最后一丝全力再试一次的冠冕堂皇的理由...,0.0,alisa,0.0,8.0,2019-02-07 02:25:00,11.0,2.729552e+08,4.0,False,False,True,False,False,False,False
2019-02-11 05:18:00,1.电影我看了，吴京炸了，京黑们快来买票参观\n2电影里年代是2075年，爷爷韩子昂是99年...,1.0,成达门业李钰霖,3.0,10.0,2019-02-11 05:18:00,9.0,1.653117e+09,2.0,False,False,True,True,False,True,False
2019-02-06 19:25:00,非常棒，科幻灾难大片的既视感，国产科幻进步非常大，构思几近满分，特效也属上乘，男主演的挺招人...,1.0,VKR802692810,0.0,10.0,2019-02-06 19:25:00,9.0,3.821338e+08,3.0,False,False,True,True,False,True,True


In [71]:
for i in data2[data2['quchuxiao']&(data2['score']>7)].nlargest(5, 'upCount')['content']:
    print(i+'\n')

前部分虽有不足，但特效是超出预期，后劲较足，到了中后部分有两个地方感动到我，刘培强实现预言，其次是王磊队长说出命令的瞬间。希望是什么？在绝境中不放弃任何一种可能，与其等待死亡，不如拼死一搏！长大又是什么？与家人共同经历之后理解彼此，打开心结。这次拯救地球集结了大家的力量，也见证了刘启的成长。流浪地球计划，让地球不再流浪。

电影删掉的部分内容:
1.妹妹想去地面，是因为想找父母的尸体，不是影片里那么中二的理由（所以有口型对不上）。2.在姥爷死了以后，兄妹二人发现冰面下全是冻着的向上游的尸体，场面很震撼。
3.王磊那么执着地去杭州，不光为了执行任务，还因为他的老婆孩子全在杭州，所以在杭州地下城毁了，火石又被周倩打坏后他才会说她老婆孩子全白死了。4.删了很多在旅途中人物的互动，比如户口跟周倩，王磊跟朵朵。特别是王磊，在失去家人后渐渐的把朵朵当成了自己女儿，这才有后面舍身救朵朵。
为了春节档排片，流浪地球前后删除了近半小时内容，因此造成了部分情节感觉不连贯。
这些全加上的话，影评中提到的所有缺点全都划掉。国产科幻的希望

“所谓常识不可逆转的时刻，只不过是人类给予自己选择不拼尽最后一丝全力再试一次的冠冕堂皇的理由。”我的观影感就是这句。
作为科幻粉，看到一部国牌的科幻大片，温情不冰冷，宏大又易懂。画面唯美得想让人瞬间上宇宙。我最喜欢“贝加尔湖的老马之约”，老马闭上眼睛接受飞向宇宙的那瞬间，泪点袭来。
本来想给💯满分，少一星只因刘启某块缺了点专业冷静。经历过骤降寒冬和大地震，他依然没意识到危机，这个片段可以理解，想人设的感情激烈，却与经历过火石培训的专业机械者背景不符，他潜意识至少应有“眼下没有退路”。幸好，在点燃火石时，专业冷静完全出来了
其实，看影评找答案，不如直接进影院感受在宇宙流浪的的科幻世界。

1.电影我看了，吴京炸了，京黑们快来买票参观
2电影里年代是2075年，爷爷韩子昂是99年生的所以他会玩手机跳海草海草。
3.引爆木星以色列科学家已经想到，但保住空间站，延续人类文明才是最佳选择，因此说成功率为零。
4.其实主角一行人没有成功过，连最后的引爆木星都是刘培强牺牲自己和空间站引爆的。
5.电影里有地面叛军与李一一的救援队枪战的情节，被剪了，所以救援队会携带武器，而刘启的救援服也是这样从黑市换来的
6.大部分的发动机分布在北半球，因此北半球是永昼区，主角才会说北

In [72]:
from pyecharts.components import Table

# 高分评论
rows_h1 = []
rows_h2 = []
rows_h3 = []
rows_h4 = []
for i in range(5):
    comment1 = [data2[data2['wumengda']&(data2['score']>7)].nlargest(5, 'upCount').values[i][0]]
    comment2 = [data2[data2['wujing']&(data2['score']>7)].nlargest(5, 'upCount').values[i][0]]
    comment3 = [data2[data2['zhaojinmai']&(data2['score']>7)].nlargest(5, 'upCount').values[i][0]]
    comment4 = [data2[data2['quchuxiao']&(data2['score']>7)].nlargest(5, 'upCount').values[i][0]]
    rows_h1.append(comment1)
    rows_h2.append(comment2)
    rows_h3.append(comment3)
    rows_h4.append(comment4)

# 低分评论
rows_l1 = []
rows_l2 = []
rows_l3 = []
rows_l4 = []
for i in range(5):
    comment1 = [data2[data2['wumengda']&(data2['score']<4)].nlargest(5, 'upCount').values[i][0]]
    comment2 = [data2[data2['wujing']&(data2['score']<4)].nlargest(5, 'upCount').values[i][0]]
    comment3 = [data2[data2['zhaojinmai']&(data2['score']<4)].nlargest(5, 'upCount').values[i][0]]
    comment4 = [data2[data2['quchuxiao']&(data2['score']<4)].nlargest(5, 'upCount').values[i][0]]
    rows_l1.append(comment1)
    rows_l2.append(comment2)
    rows_l3.append(comment3)
    rows_l4.append(comment4)

In [73]:
c = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add_xaxis(['郭帆', '李光洁', '刘慈欣', '屈楚萧', '吴京', '吴孟达','赵今麦'])
    .add_yaxis("好评提及次数", list1, stack="stack1", category_gap="50%")
    .add_yaxis("差评提及次数", list2, stack="stack1", category_gap="50%")
    .add_yaxis("中评提及次数", list3, stack="stack1", category_gap="50%")
    .set_series_opts(
        label_opts=opts.LabelOpts(
            position="right",
            formatter=JsCode(
                "function(x){return Number(x.data.percent * 100).toFixed() + '%';}"
            ),
        )
    )
)
c.render("./Charts_html/Evaluate_bar.html")


'D:\\CODE\\demo_py\\Project\\coursedesign_python\\Django\\Data_Analysis\\Charts_html\\Evaluate_bar.html'

In [74]:
# 以下的表格画出来不好用
# 画好评差评中评在单条的条形图
def evaluate_bar() -> Bar:

    c = (
        Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
        .add_xaxis(['郭帆', '李光洁', '刘慈欣', '屈楚萧', '吴京', '吴孟达','赵今麦'])
        .add_yaxis("好评提及次数", list1, stack="stack1", category_gap="50%")
        .add_yaxis("差评提及次数", list2, stack="stack1", category_gap="50%")
        .add_yaxis("中评提及次数", list3, stack="stack1", category_gap="50%")
        .set_series_opts(
            label_opts=opts.LabelOpts(
                position="right",
                formatter=JsCode(
                    "function(x){return Number(x.data.percent * 100).toFixed() + '%';}"
                ),
            )
        )
    )
    return c


# 画高分评论表格
def table_baseh1() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h1

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="吴孟达")
    )
    return table

def table_baseh2() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h2

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="吴京")
    )
    return table

def table_baseh3() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h3

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="赵今麦")
    )
    return table

def table_baseh4() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h4

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="屈楚萧")
    )
    return table


# 画高分评论表格
def table_basel1() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h1

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="吴孟达")
    )
    return table

def table_basel2() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h2

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="吴京")
    )
    return table

def table_basel3() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h3

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="赵今麦")
    )
    return table

def table_basel4() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h4

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="屈楚萧")
    )
    return table



def page_layout():
    page = Page(layout=Page.SimplePageLayout)
    page.add(
        evaluate_bar(),
        table_baseh1(),
        table_baseh2(),
        table_baseh3(),
        table_baseh4(),
        table_basel1(),
        table_basel2(),
        table_basel3(),
        table_basel4(),


    )
    page.render("evaluate_bar_table.html")


if __name__ == "__main__":
    page_layout()

In [75]:
temp = data2[data2['zhaojinmai']&(data2['score']>7)].nlargest(5, 'upCount').values[0][0]
print(temp)
# for i in
comment = []
for i in range(5):
    temp= [data2[data2['zhaojinmai']&(data2['score']>7)].nlargest(5, 'upCount').values[i][0]]
    comment.append(temp)

print(comment)

世界观宏大，特效过关，节奏紧凑。
就是剧情略有欠缺俗套，改编的不能说很成功，从人类大局观来说，剧情的改编多了太多感性，人类在危机纪元不应该如此感性，不太理智，埋没了大刘对于全人类共同对抗危机时自我牺牲的精神，这点扣分。
男女主演技还是略年轻。

整体感觉还不错，推荐
[['世界观宏大，特效过关，节奏紧凑。\n就是剧情略有欠缺俗套，改编的不能说很成功，从人类大局观来说，剧情的改编多了太多感性，人类在危机纪元不应该如此感性，不太理智，埋没了大刘对于全人类共同对抗危机时自我牺牲的精神，这点扣分。\n男女主演技还是略年轻。\n\n整体感觉还不错，推荐'], ['猪年零点首映场观影值了，看完有点打鸡血式兴奋！本片开启中国科幻片里程碑小马达，整体节奏不错，特效和音乐很棒，片尾曲好听，女主有点傻傻萌，她的煽情台词稍显啰嗦，还有几帧场景宣誓式的台词是硬伤，故事情节大众，总体fine还不赖的一部作品能优点盖过缺点，预测猪年贺岁档黑马一匹！看这类特效影片就得必选IMAX才能完爆体验。'], ['非常棒，科幻灾难大片的既视感，国产科幻进步非常大，构思几近满分，特效也属上乘，男主演的挺招人烦，女主镜头不多但很漂亮，吴京达叔演技还是杠杠滴，非要给意见，那就是如果把木星波推动地球那一瞬间给个镜头绝对震撼。'], ['原来流浪地球还有145分钟和160分钟的版本，其实里面有很多挺重要的部分呀\n比如王磊的妻子孩子其实就住在杭州地下城，所以他才要拼了命都要去杭州救他们\n韩朵朵出去其实是为了找自己亲生父母的尸体，然后姥爷死了之后韩朵朵和刘启找到了朵朵被救的地方，然后有个特效就是她跪下，然后用手把冰面擦开，冰下全是向上游的尸体。\n导演我只想说，我想看160分钟完整版！！！'], ['这是我24年人生看过特效最真实最震撼的国产科幻电影。\n唯一扣分项就是女主韩朵朵，我感觉这个角色太多余了，只会大喊大叫。没有动力甲去退“撞针”还占一个使力的位置。特别广播那一段特别尴尬，干嘛不让联合政府来广播？让一个小女孩来，还讲一段故事煽情一下😅有些老套，木星撞地球啊可是亿万火急的呀。 \nPS:奥观海同志在哪啊？没看见他🤣。\nPS2：吴京撞木星上先把冬眠的生活仓环带脱离了，并不是带着一帮宇航员一起死。  \nPS3:地面还可能有幸存者。']]
